# Parameter estimation

`DiffEqParamEstim.jl` is not installed with `DifferentialEquations.jl`. You need to install it manually:

```julia
using Pkg
Pkg.add("DiffEqParamEstim")
using DiffEqParamEstim
```

## Estimate a single parameter from the data and the ODE model

Let's optimize the parameters of the Lotka-Volterra equation.

In [ ]:
using DifferentialEquations
using Plots
using DiffEqParamEstim
using Optim

In [ ]:
function lotka_volterra!(du, u, p, t)
    du[1] = dx = p[1]*u[1] - u[1]*u[2]
    du[2] = dy = -3*u[2] + u[1]*u[2]
end

In [ ]:
u0 = [1.0;1.0]
tspan = (0.0, 10.0)
p = [1.5]
prob = ODEProblem(lotka_volterra!, u0, tspan, p)
sol = solve(prob, Tsit5())

ts = range(tspan[begin], stop=tspan[end], length=200)

We build a sample dataset with some noise.

In [ ]:
data = [sol.(ts, idxs=1) sol.(ts, idxs=2)] .* (1 .+ 0.03 .* randn(length(ts), 2))

In [ ]:
plot(sol)
scatter!(ts, data, label=["u1 data" "u2 data"])

Use `build_loss_objective()` to build a loss function for the ODE problem with the data. We will minimize the mean squared error using `L2Loss()`. Note that the data should be transposed.

In [ ]:
alg = Tsit5()
cost_function = build_loss_objective(prob, alg, L2Loss(collect(ts), transpose(data)), maxiters=10000, verbose=false)

plot(cost_function, 0.0, 10.0,
     linewidth = 3, label=false, yscale=:log10,
     xaxis = "Parameter", yaxis = "Cost", title = "1-Parameter Cost Function"
)

There is a dip (minimum) in the cost function at the true parameter value (1.5).
We can use an optimizer, e.g., `Optim.jl`, to find the parameter value that minimizes the cost.

In [ ]:
result = Optim.optimize(cost_function, 0.0, 10.0)

In [ ]:
result.minimizer

We have recovered the true parameter of 1.5!

You can also find the parameter with a initial guess using the `BFGS()` / `LBFGS()` method. Tha latter records less trajectories and thus uses less memory.

In [ ]:
result = optimize(cost_function, [1.42], LBFGS())

In [ ]:
# Note that the result is a vector because we used a different optimization algorithm
result.minimizer

If you have a pair of lower and upper bounds, you can use the `Fminbox()` method.

In [ ]:
result = optimize(cost_function, [0.0], [3.0], [1.42], Fminbox(BFGS()))

In [ ]:
# Note that the result is a vector
result.minimizer

## Estimate multiple parameters

Let's use the Lotka-Volterra (Fox-rabbit) equations with all 4 parameters free.

In [ ]:
function f2(du, u, p, t)
    du[1] = dx = p[1]*u[1] - p[2]*u[1]*u[2]
    du[2] = dy = -p[3]*u[2] + p[4]*u[1]*u[2]
end
  
u0 = [1.0; 1.0]
tspan = (0.0, 10.0)
p = [1.5, 1.0, 3.0, 1.0]
prob = ODEProblem(f2, u0, tspan, p)
sol = solve(prob, Tsit5())

ts = range(tspan[begin], stop=tspan[end], length=200)
data = [sol.(ts, idxs=1) sol.(ts, idxs=2)] .* (1 .+ 0.01 .* randn(length(ts), 2))

Then we can find multiple parameters at once using the same procedure.

In [ ]:
cost_function = build_loss_objective(prob, Tsit5(), L2Loss(collect(ts), transpose(data)), maxiters=10000, verbose=false)
result_bfgs = Optim.optimize(cost_function, [1.3, 0.8, 2.8, 1.2], LBFGS())

In [ ]:
# True parameters: [1.5, 1.0, 3.0, 1.0]
result_bfgs.minimizer